In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

data = pd.read_csv('Training_data/MLIN_1_2_mid.csv')
data_int=data.iloc[:,:]
print(data_int.shape)
data_int_10=np.tile(data_int, (30, 1)) #串聯10倍長度
mean,var = 0,0.001
noise = np.random.normal(mean, var, data_int_10.shape[0]) #並加入noise
noise = np.random.normal(mean, var, data_int_10.shape) #並加入noise
print(noise.shape)

data_noise_10 = data_int_10+noise*0.00001 #嘗試在不同大小的noise

X = data_noise_10[:, :2]
y = data_noise_10[:, 2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)



(1701, 7)
(51030, 7)
(40824, 2) (10206, 2) (40824, 5) (10206, 5)


In [ ]:
xgb = XGBRegressor(objective='reg:squarederror',
                   seed=42)
param_grid = {
    'estimator__learning_rate': [0.01,0.05,0.1],
    'estimator__n_estimators': [800,900,1000],
    'estimator__max_depth': [7,8,9,10],
       'estimator__min_child_weight': [5 , 7],
       'estimator__gamma': [0.0,0.1,0.2],
       'estimator__subsample': [0.8, 0.9 , 1]
}
# 
multi_output_model = MultiOutputRegressor(xgb)

grid_search = GridSearchCV(estimator=multi_output_model,
                           param_grid=param_grid,
                           cv = 2,
                           scoring='neg_mean_squared_error',
                           verbose=1,
                           n_jobs=-1
                          )

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

best_xgb_model = XGBRegressor(objective='reg:squarederror',
                              learning_rate=best_params['estimator__learning_rate'],
                              n_estimators=best_params['estimator__n_estimators'],
                              max_depth=best_params['estimator__max_depth'],
                               min_child_weight=best_params['estimator__min_child_weight'],
                               gamma=best_params['estimator__gamma'],
                               subsample=best_params['estimator__subsample'],
                              colsample_bytree=0.8,
                              reg_alpha=0,
                              reg_lambda=1,
                              seed=42)
best_multi_output_model = MultiOutputRegressor(best_xgb_model)

best_multi_output_model.fit(X_train, y_train)

y_pred = best_multi_output_model.predict(X_test)
# 
r2_scores = r2_score(y_test, y_pred, multioutput='raw_values')
print(f'R² scores for each output: {r2_scores}')

Fitting 2 folds for each of 648 candidates, totalling 1296 fits


In [ ]:
RMSE=np.sqrt(sum((y_test-y_pred) ** 2)/(y_pred.size))
print('RMSE [S11,S21,beta,alpha,Zo] : ')#,S11(angle),S21(angle)
print(RMSE)
NRMSE=RMSE/abs(np.mean(y_pred, axis=0))
print('NRMSE [S11,S21,beta,alpha,Zo] :')# ,S11(angle),S21(angle)
print(NRMSE)